In [2]:
from random import * # random numbers
import os, subprocess # to check and create directories 
import math # python math
import numpy as np # numerical python
import scipy # scientific python
from scipy import optimize # for numerical solution of equations
from matplotlib import pyplot as plt # plotting
import matplotlib.gridspec as gridspec # more plotting 
from prettytable import PrettyTable # pretty printing of tables
from tqdm import tqdm # display progress
from optparse import OptionParser # command line parameters
from scipy.integrate import quad # for numerical integrals
from matplotlib.ticker import MultipleLocator
from alphaS import * # for the alphaS calculation



In [3]:
# initialize alphaS: pass the value of alphaS at mz, and mz, creo un oggetto della classe alphaS
aS = alphaS(0.118, 91.1876)

Let's now define some switches that will allow us to vary the evolution starting and cutoff scales, as well as whether to print debugging information.

```Qc``` and ```Q``` are the cutoff and "hard" scales, respectively. 

```Nevolve``` represents the number of "evolutions" to perform in this simple example. Note that kinematics will not be reconstructed in this warmup exercise.

In [4]:
# RUN OPTIONS
# the number of branches to evolve:
Nevolve = 100000
# the cutoff scale, e.g. 1 GeV. 
Qc = 1.
# the hard scale, e.g. 1000 GeV
Q = 1000.



Ora scegliamo in che modo vogliamo trattare la scala di alphaS. Una scala fissa può essere preferibile in certi casi per esempio per confrontare la splitting function con la distribuzione di z ottenuta dalla simulazione.

In [5]:
# fixed scale if the alphaS is fixed:
scaleoption = "pt" # "pt" for the default scale, "fixed" for a fixed scale, given by fixedScale:
fixedScale = Q/2. # if the choice above is "fixed"


Let's now define:

- the splitting function, $P(z) = C_F \frac{1+z^2}{1-z}$, given by ```Pqq(z)```,
- the overestimate of the splitting function $\hat{P} = C_F\frac{2}{1-z}$, given by ```Pqq_over(z)```,
- a function that returns the scale of $\alpha_S$, according to the global variable ```scaleoption```, ```scale_of_alphaS```,
- the function that returns $\alpha_S$, accessed through ```LHAPDF```, ```alphaS```,
- the function $t \times \hat{\Gamma}(z) = \int_{\hat{z}_-}^{\hat{z}^+} \mathrm{d} z \frac{\hat{\alpha_S}}{2\pi} \hat{P}(z)=\int_{\hat{z}_-}^{\hat{z}^+} \mathrm{d} z \frac{\hat{\alpha_S}}{2\pi} C_F \frac{2}{1-z} =\frac{\hat{\alpha_S}}{2\pi} 2 C_F \log(\frac{1-\hat{z}^-}{1+\hat{z}^+})$, given by ```tGamma```.

  In realtà tGamma mi da $t \times \hat{\Gamma}(z) = \int_{}^{z} \mathrm{d} z' \frac{\hat{\alpha_S}}{2\pi} \hat{P}(z')=\int_{}^{z} \mathrm{d} z' \frac{\hat{\alpha_S}}{2\pi} C_F \frac{2}{1-z'}=-\frac{\hat{\alpha_S}}{2\pi} 2 C_F \log(1-z)$
- the inverse of the function $t \times \hat{\Gamma}(z)$, given by ```inversetGamma```,
- the overestimates of the limits of integration $\hat{z}_-$ and $\hat{z}_+$, given by ```zm_over``` and ```zp_over```:

  $\hat{z}_+(t)=1-\sqrt{\frac{Q_c^2}{t}}$

  $\hat{z}_-(t)=\sqrt{\frac{Q_c^2}{t}}$
- and the function that gives the overestimate of $\alpha_S$, ```get_alphaS_over```. 

In [6]:
# the q -> q + g splitting function
def Pqq(z): return CF * (1. + z**2)/(1.-z)
    
# the q -> q + g splitting function *overestimate* 
def Pqq_over(z): return 2.*CF/(1.-z)


# return the true alphaS using the PDF alphaS over 2 pi

def alphaS(t, z, Qcut, aSover):

    if scaleoption == "pt":
        scale= z * (1-z) * np.sqrt(t)
    elif scaleoption == "fixed":
        scale= fixedScale
    
    if scale < Qc:
        return aS.alphasQ(Qcut)/2./math.pi
    
    return aS.alphasQ(scale)/2./math.pi


# set the overestimate of alphaS once and for all
def get_alphaS_over(Q, Qcut):
    
    if scaleoption == "pt":
        minscale = Qcut # the minimum scale^2 available to the PDF
        scale = minscale
    elif scaleoption == "fixed":
        scale = fixedScale
    alphaS_over = aS.alphasQ(scale)/2./math.pi
    
    return alphaS_over

# l'integrale di tGamma

def tGamma(z, aSover):
    return -2.*aSover*CF*np.log(1-z) 

# the inverse of the function t*Gamma, given the overestimate for alphaS:

def inversetGamma(r, aSover):
    return 1. - np.exp(- 0.5*r/CF/aSover)

# the overestimated upper and lower limit for the z integral:

def zp_over(t, Qcut): 
    return 1.-np.sqrt(Qcut**2/t)
def zm_over(t, Qcut): 
    return np.sqrt(Qcut**2/t)



# get the momentum fraction candidate for the emission

def Get_zEmission(t, Qcut, R, aSover): 
    return inversetGamma( tGamma(zm_over(t, Qcut), aSover) + R * ( tGamma(zp_over(t, Qcut), aSover) - tGamma(zm_over(t, Qcut), aSover)), aSover)

# calculate the transverse momentum of the emission
def Get_pTsq(t, z): return z**2 * (1-z)**2 * t

# calculate the virtual mass-squared of the emitting particle
def Get_mvirtsq(t,z): return z*(1-z) * t



Given the above, the treatment now proceeds as follows:

The overstimate of the Sudakov form factor is given by:

$\hat{\Delta}(t, t_\mathrm{max}) = \exp\left\{ - \int^{t_\mathrm{max}}_{t} \mathrm{d}t' \hat{\Gamma} (t') \right\}$,

with:

$\hat{\Gamma} (t') = \frac{1}{t'}   \int_{\hat{z}_-}^{\hat{z}^+} \mathrm{d} z \frac{\hat{\alpha_S}}{2\pi} \hat{P}(z)$.

We are working with the overestimated functions, and this allows us to analytically solve:

$\hat{\Delta}(t, t_\mathrm{max}) = R$, 

where $R$ is a random number in $[0,1]$. 

Therefore, we are seeking the solution of:

$\exp\left\{ - \int^{t_\mathrm{max}}_{t} \mathrm{d}t' \hat{\Gamma} (t') \right\} = R$,

or:

$\int^{t_\mathrm{max}}_{t} \mathrm{d}t' \hat{\Gamma} (t') = \ln\left(\frac{1}{R}\right)$.

Definiamo la funzione  $\rho({\hat{z(t')}_+},{\hat{z(t')}_-}) = \int_{\hat{z}_-}^{\hat{z}^+} \mathrm{d} z \frac{\hat{\alpha_S}}{2\pi} \hat{P}(z) = t'\hat{\Gamma} (t') $, to obtain:

$\int^{t_\mathrm{max}}_{t} \frac{\mathrm{d}t'}{t'} \rho(t') = \ln\left(\frac{1}{R}\right)$, 

or:

$\int^{t_\mathrm{max}}_{t'} \mathrm{d}\ln t' \rho(t) = \ln\left(\frac{1}{R}\right)$.

If the limit overestimates in $\rho({\hat{z}_+},{\hat{z}_-})$ do not depend on $t'$, then: 

$\left. \rho \ln t \right|_t^{t_\mathrm{max}} = \ln\left(\frac{1}{R}\right) \Rightarrow \rho \ln \frac{t_\mathrm{max}}{t}  = \ln\left(\frac{1}{R}\right) $, 

which yields after exponentiation:

$t =  t_\mathrm{max} R^{1/\rho}$.

In the numerical evaluation of $t$, we instead focus on:

$\ln \frac{t}{t_\mathrm{max}} = \frac{1}{\rho} \ln R$,

and construct the function:

$E(t) = \ln \frac{t}{t_\mathrm{max}} - \frac{1}{\rho} \ln R$,

where now, explicitly, $\rho({\hat{z}_+},{\hat{z}_-}) = \int_{\hat{z}_-}^{\hat{z}_+} \mathrm{d} z \frac{\hat{\alpha_S}}{2\pi} \hat{P}(z) = \int^{\hat{z}_+} \mathrm{d} z \frac{\hat{\alpha_S}}{2\pi} \hat{P}(z) - \int^{\hat{z}_-} \mathrm{d} z \frac{\hat{\alpha_S}}{2\pi} \hat{P}(z)$.

The function ```EmissionScaleFunc``` in the code that follows corresponds to $E(t)$ and the $\rho$ function is calculated via: ```r = tGamma(zp_over(t, Qc), aSover) - tGamma(zm_over(t, Qc), aSover)```. 

The "next" value of $t$ is then calculated by solving $E(t)=0$ in the function ```Get_tEmission```. This is done numerically via ```scipy.optimize.ridder```. 

The ```Get_tEmission``` function actually finds the value of $\ln(t/t_\mathrm{max})$ for which the solution is obtained, and therefore needs to be exponentiated and multiplied by $t_\mathrm{max}$ (represented by ```Q**2```) to get the value of $t$. 

In [7]:
# the function E(ln(t/Q**2)) = ln(t/Q**2) - (1/r) ln(R) for the numerical solution for the evolution scale, given random number R

def EmissionScaleFunc(logt_over_Qsq, Q, Qcut, R, aSover):
    # calculate t:
    t = Q**2 * np.exp( logt_over_Qsq )
    # get r:
    r = tGamma(zp_over(t, Qcut), aSover) - tGamma(zm_over(t, Qcut), aSover)

    # calculate E(ln(t/Q**2)), the equation to solve

    return logt_over_Qsq - (1./r) * np.log(R)

# una funzione che calcola numericamente la scala di emissione t dato la scala iniziale Q (il t da cui si parte), il cutoff Qc e un numero casuale R
def Get_tEmission(Q, Qcut, R, tfac, aSover):

    tolerance = 1E-3 # the tolerance for the solution

    EmissionFunc_arg = lambda x : EmissionScaleFunc(x, Q,Qcut,R,aSover) # the function in a form appropriate for the solver

    # calculate the solution using "Ridder's" method

    lower_bound=np.log(tfac*Qcut**2/Q**2)
    upper_bound=0.

    sol, results = scipy.optimize.ridder(EmissionFunc_arg, lower_bound, upper_bound, xtol=tolerance, full_output=True, maxiter=1000)


    # get the actual evolution variable from the solution
    tEm = Q**2 * np.exp( sol )
   

    # if a solution has not been found, terminate the evolution        
    if abs(EmissionFunc_arg(sol)) > tolerance:
            return Q**2, [], False

    # otherwise return the emission scale and continue
    return tEm, results, True

In [8]:
Get_tEmission(800,1,0.95,3.99,0.015)

(526553.5028254912,
       converged: True
            flag: 'converged'
  function_calls: 8
      iterations: 3
            root: -0.19511523014112675,
 True)

To get the $z$ variable, we need to solve:

$ \rho (z,{\hat{z}_-}) = R' \rho({\hat{z}_+},{\hat{z}_-})$,

where $R'$ is another random number in $[0,1]$. 

Recalling that: 

$t' \hat{\Gamma} (t') = \int_{\hat{z}_-}^{\hat{z}_+} \mathrm{d} z \frac{\hat{\alpha_S}}{2\pi} \hat{P}(z)$, 

we now define

$\tilde{\rho}(z) = \int^{z} \mathrm{d} z \frac{\hat{\alpha_S}}{2\pi} \hat{P}(z)$, we can write the equation for $z$ as:

$\tilde{\rho}(z) - \tilde{\rho}({\hat{z}_-}) = R' \left[ \tilde{\rho}({\hat{z}_+}) - \tilde{\rho}({\hat{z}_-}) \right]$,

and we obtain $z$ by solving:

$ z = \tilde{\rho}^{-1} \left\{ \tilde{\rho}({\hat{z}_-}) + R' \left[\tilde{\rho}({\hat{z}_+}) - \tilde{\rho}({\hat{z}_-}) \right] \right\}$,

where $\tilde{\rho}^{-1}$ is the inverse of $\tilde{\rho}$, which is given by the ```inversetGamma``` function in our code. 

The following function ```Generate_Emission``` solves for the next $t$ and $z$ using the overestimates, and then implements the vetoing by constructing a probability with the full splitting function and value of $\alpha_S$. 

In [9]:
# function that generates emissions:
def Generate_Emission(Q, Qcut, tfac, aSover):
    generated = True
    # generate random numbers
    R1 = random()
    R2 = random()
    R3 = random() #veto test over z
    R4 = random() #veto test
    # solve for the (candidate) emission scale:

    tEm, results, continueEvolution = Get_tEmission(Q, Qcut, R1, tfac, aSover)
    

    # if no solution is found then end branch

    if continueEvolution == False:
        zEm = 1.
        pTsqEm = 0.
        MsqEm = 0.
        return tEm, zEm, pTsqEm, MsqEm, generated, continueEvolution

    
    
    # get the (candidate) z of the emission
    zEm = Get_zEmission(tEm, Qc, R2, aSover)
    
    # get the transverse momentum 
    pTsqEm = Get_pTsq(tEm, zEm)

    

    # now check the conditions to accept or reject the emission:
    # check if the transverse momentum is physical:
    if pTsqEm < 0.:
      
        generated = False
    # compare the splitting function overestimate prob to a random number
    if Pqq(zEm)/Pqq_over(zEm) < R3:
        generated = False
    
    # compare the alphaS overestimate prob to a random number
    if alphaS(tEm, zEm, Qc, aSover)/aSover < R4:
        generated = False
    
    # get the virtual mass squared:
    MsqEm = Get_mvirtsq(tEm, zEm)
    if generated == False: # rejected emission
        zEm = 1.
        pTsqEm = 0.
        MsqEm = 0.
    # return all the variables for the emission
    return tEm, zEm, pTsqEm, MsqEm, generated, continueEvolution

The above function generates the next emission scale and momentum fraction. To get the full evolution, the main function uses it, until the cutoff scale is reached. 

Note that the next evolution scale is set to $\tilde{t} z^2$ in order to implement angular ordering in the branchings

In [10]:
# the function that performs the evolution of a single branch:
# returns a list of all the emissions for further processing

def Evolve(Q, Qmin, aSover):
    # the minimum evolution scale
    tEm_min = Qmin**2
    # counter for the number of emissions:
    Nem = 0
    # array to store emission info:
    Emissions = []
    fac_tEm = 3.999 # minimum value for the cutoff to try emissions = fac_tEm * Qc**2 (should be less than the actual cutoff)
    fac_cutoff = 4. # actual cutoff = fac_cutoff * Qc**2

    # start the evolution

    tEm = Q**2 # initial value of the evolution variable
    zEm = 1 # initial value of the momentum fraction

    

    while np.sqrt(tEm)*zEm > np.sqrt(fac_cutoff*tEm_min):
        # evolve:
        tEm, zEm, pTsqEm, MsqEm, generatedEmission, continueEvolution = Generate_Emission(np.sqrt(tEm)*zEm, np.sqrt(tEm_min), fac_tEm, aSover)

        # if the solver could not find a solution, end the evolution
        if continueEvolution == False:
           return Emissions
        # if we have already passed the cutoff this emission does not count
        # this will also terminate the evolution
        if tEm < fac_cutoff*tEm_min: 
           
            zEm = 1.
            pTsqEm = 0.
            QsqEm = 0.
        
            return Emissions

        # if the emission was successful, append to the Emissions list and continue
        if zEm != 1.0:
            
            Emissions.append([math.sqrt(tEm), zEm, math.sqrt(pTsqEm), math.sqrt(MsqEm)])
            Nem = Nem + 1
   
    return Emissions


    





In [38]:

Evolve(502,1,0.015)



[[308.252621265326, 0.9811581306815169, 5.698620982220289, 41.91198939882228],
 [164.13872311330437,
  0.8321169787796813,
  22.929902619446466,
  61.348878857468584]]

We are now able to perform the evolution! Note that this corresponds to the evolution of a single "quark" line in its center-of-mass frame. Therefore, no full reconstruction is performed here. 

In [12]:
##########################
# Evolution begins here! #
##########################

# set the overestimate of alphaS once and for all:
alphaS_over = get_alphaS_over(Q, Qc)
print(alphaS_over)





    
# list to store all emission information:
AllEmissions = []

print('Evolving', Nevolve, 'branches from Q=', Q, 'GeV --> Qc=', Qc, 'GeV')

# perform evolution over Nevolve branches:
for j in tqdm(list(range(Nevolve))): # tqdm is the progress bar 
    # perform the evolution over a single branch j and return a list of all the emissions:
    Emissions = Evolve(Q,Qc,alphaS_over)
    # concatenate the emissions of this evolution branch to the list of all emissions to plot
    AllEmissions = AllEmissions + Emissions

print('All evolutions ended')


0.06992354855453746
Evolving 100000 branches from Q= 1000.0 GeV --> Qc= 1.0 GeV


  9%|▊         | 8745/100000 [00:14<02:26, 621.68it/s]


KeyboardInterrupt: 

Let's compare the momentum fraction with the expected one from the full splitting function. Note that this will only be exact if the scale has been set to fixed, otherwise there will be some dependence of $\alpha_S$ on $z$ by using the $p_T$ of the emissions as its argument. 

In [ ]:
len(AllEmissions)

In [ ]:
###########################
print('---')
print('plotting z of emissions')
# plot settings ########
plot_type = 'momentumfrac'
# plot:
# plot settings
ylab = '$(1-z)P(z)$'
xlab = '$z$'
ylog = False
xlog = False
nbins=40
# construct the axes for the plot
fig = plt.figure(constrained_layout=True)

gs = gridspec.GridSpec(4, 4,figure=fig,wspace=0, hspace=0)
ax = fig.add_subplot(gs[:3, :])
ax2 = fig.add_subplot(gs[3, :])
gs.update(wspace=0,hspace=0)

ax.grid(False)
ax2.xaxis.set_minor_locator(MultipleLocator(0.05))
ax2.yaxis.set_minor_locator(MultipleLocator(0.025))

tarray = []
for i in range(len(AllEmissions)):
    tarray.append(np.array(AllEmissions[i][1]))

gs.update(wspace=0.0, hspace=0.0)

# get the histogram bins:
bins, edges = np.histogram(tarray, bins=nbins, density=True)

left,right = edges[:-1],edges[1:]
X = np.array([0.5*left+0.5*right]).T.flatten()
Y = np.array([bins]).T.flatten() * (1-X)
# normalise:
xnorm_min=0.0
xnorm_max=1.0

#Y = Y/np.linalg.norm(Y)/(Y[1]-Y[0])
#Ysum = Y[(X>xnorm_min) & (X<xnorm_max)].sum()
gs.update(wspace=0.0, hspace=0.0)


# compare to the input splitting function
# this comparison is only correct if alphaS is fixed
# this is because the scale of alphaS is also a function of z 
Yspl = Pqq(X) * (1-X)

# get the integral numerically, but not in the whole range
# since the splitting function diverges as z->1 and this cannot be captured numerically:
zp = X[(X<xnorm_max)][-1]
zm = X[(X>xnorm_min)][0]
def Pqq1mz(z):
    return Pqq(z) * (1-z)
YsplI = quad(Pqq1mz, 0, 1)
YsplI = np.linalg.norm(Yspl)
print(np.linalg.norm(Yspl))
ax.plot(X,Yspl, color='blue', lw=1, label='Splitting function', marker='x', ms=0)

# plot:
print(np.linalg.norm(Y))
Y = YsplI * Y / np.linalg.norm(Y)
print(np.linalg.norm(Y))
print(YsplI)

ax.plot(X,Y, label='Pyresias', color='red', lw=0, marker='o', ms=2)


# ratio:
ax2.plot(X,Y/Yspl, color='red', lw=0, label='Splitting function', marker='o', ms=2)
ax2.hlines(y=1, xmin=0, xmax=1, color='black', ls='--')

# set the ticks, labels and limits etc.
ax.set_ylabel(ylab, fontsize=20)
ax2.set_xlabel(xlab, fontsize=20)
ax2.set_ylabel('Pyr./Spl.')
ax2.set_ylim(0.9,1.1)
ax2.set_xlim(0.0,1.0)
ax.set_xlim(0.0,1.0)
# choose x and y log scales
if ylog:
    ax.set_yscale('log')
else:
    ax.set_yscale('linear')
if xlog:
    ax.set_xscale('log')
else:
    ax.set_xscale('linear')

# create legend and plot/font size
ax.legend()
ax.legend(loc="upper center", numpoints=1, frameon=False, prop={'size':10})
ax.set_xticklabels('')
ax.set_xticks([])
infile = plot_type + '.dat'
plt.savefig('plots/' + infile.replace('.dat','.pdf'), bbox_inches='tight')


In [ ]:
plt.hist(tarray, bins=nbins, density=True)


In [ ]:
np.histogram(tarray, bins=nbins, density=True)
len(tarray)

In [ ]:
###########################
print('---')
print('plotting z of emissions')
# plot settings ########
plot_type = 'momentumfrac'
# plot:
# plot settings
ylab = '$P(z)$'
xlab = '$z$'
ylog = False
xlog = False
nbins=40
# construct the axes for the plot
fig = plt.figure(constrained_layout=True)

gs = gridspec.GridSpec(4, 4,figure=fig,wspace=0, hspace=0)
ax = fig.add_subplot(gs[:3, :])
ax2 = fig.add_subplot(gs[3, :])
gs.update(wspace=0,hspace=0)

ax.grid(False)
ax2.xaxis.set_minor_locator(MultipleLocator(0.05))
ax2.yaxis.set_minor_locator(MultipleLocator(0.025))

tarray = []
for i in range(len(AllEmissions)):
    tarray.append(np.array(AllEmissions[i][1]))

gs.update(wspace=0.0, hspace=0.0)

# get the histogram bins:
bins, edges = np.histogram(tarray, bins=nbins, density=True)

left,right = edges[:-1],edges[1:]
X = np.array([0.5*left+0.5*right]).T.flatten()
Y = np.array([bins]).T.flatten() 
# normalise:
xnorm_min=0.0
xnorm_max=1.0

#Y = Y/np.linalg.norm(Y)/(Y[1]-Y[0])
#Ysum = Y[(X>xnorm_min) & (X<xnorm_max)].sum()
gs.update(wspace=0.0, hspace=0.0)


# compare to the input splitting function
# this comparison is only correct if alphaS is fixed
# this is because the scale of alphaS is also a function of z 
Yspl = Pqq(X) 

# get the integral numerically, but not in the whole range
# since the splitting function diverges as z->1 and this cannot be captured numerically:
zp = X[(X<xnorm_max)][-1]
zm = X[(X>xnorm_min)][0]

YsplI = quad(Pqq, 0, 0.9999)
YsplI = np.linalg.norm(Yspl)
print(np.linalg.norm(Yspl))
ax.plot(X,Yspl, color='blue', lw=1, label='Splitting function', marker='x', ms=0)

# plot:
print(np.linalg.norm(Y))
Y = YsplI * Y / np.linalg.norm(Y)
print(np.linalg.norm(Y))
print(YsplI)

ax.plot(X,Y, label='Pyresias', color='red', lw=0, marker='o', ms=2)


# ratio:
ax2.plot(X,Y/Yspl, color='red', lw=0, label='Splitting function', marker='o', ms=2)
ax2.hlines(y=1, xmin=0, xmax=1, color='black', ls='--')

# set the ticks, labels and limits etc.
ax.set_ylabel(ylab, fontsize=20)
ax2.set_xlabel(xlab, fontsize=20)
ax2.set_ylabel('Pyr./Spl.')
ax2.set_ylim(0.9,1.1)
ax2.set_xlim(0.0,1.0)
ax.set_xlim(0.0,1.0)
# choose x and y log scales
if ylog:
    ax.set_yscale('log')
else:
    ax.set_yscale('linear')
if xlog:
    ax.set_xscale('log')
else:
    ax.set_xscale('linear')

# create legend and plot/font size
ax.legend()
ax.legend(loc="upper center", numpoints=1, frameon=False, prop={'size':10})
ax.set_xticklabels('')
ax.set_xticks([])
infile = plot_type + '.dat'
plt.savefig('plots/' + infile.replace('.dat','.pdf'), bbox_inches='tight')